# Problema 1) 

Usando dados da economia norte-americana estime o modelo de Campbell e Mankiw (1989), assim descrito:

$$\Delta C_{t+1} = \tilde{\tau}_0 + \lambda \Delta \ln Y_{t+1} + \tilde{\tau}_1 r_{t+1} + \tilde{\varepsilon}_{t+1}$$

Considere duas frequências: i) trimestral; ii) anual.

Considere duas medidas diferentes para bens não duráveis: i) gasto com bens não duráveis; ii) gasto com bens não duráveis e serviços.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from statsmodels.sandbox.regression import gmm

from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', RuntimeWarning)
warnings.filterwarnings('ignore', 'Optimization terminated successfully*')

In [ ]:
# Importando os dados do Github
df_a  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/dataset_anual.csv',
                    delimiter = ",", index_col=0
                    )

df_q  = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/dec_consumo_poupanca/main/dados/dataset_trim.csv',
                    delimiter = ",", index_col=0
                    )

# Ajustado o indice como data
df_q.index = pd.to_datetime(df_q.index)
df_a.index = pd.to_datetime(df_a.index)

df_a.head(n = 5)

,pib,c,c_growth,pc,k,k_growth,pk,s,s_growth,ps,sp500_ret_real,TBill_ret_real
1954-01-01,26781.114997,6083.636364,0.015872,16.775,645.756316,-0.004260,52.207,10162.654140,0.034690,10.218,-0.056324,0.013250
1955-01-01,28767.951318,6399.435977,0.051910,16.666,783.482357,0.213279,52.005,10671.735155,0.050093,10.391,0.481662,0.023219
1956-01-01,29377.157427,6624.371488,0.035149,16.905,754.219944,-0.037349,53.259,11183.775146,0.047981,10.638,0.208329,0.009069
1957-01-01,29943.718022,6757.183908,0.020049,17.400,761.096724,0.009118,55.219,11571.193716,0.034641,10.949,-0.008021,0.000871
1958-01-01,29707.327613,6846.835727,0.013268,17.824,703.105546,-0.076194,56.222,11946.345811,0.032421,11.220,-0.146485,-0.004951


In [ ]:
df_q.head(n = 5)

,pib,c,c_growth,pc,k,k_growth,pk,s,s_growth,ps,sp500_ret_real,TBill_ret_real
1954-01-01,26456.917267,6032.108179,0.009646,16.787,612.575385,-0.030679,53.724,9916.723952,0.009418,10.183,0.054588,-0.001841
1954-04-01,26461.143092,6016.150101,-0.002646,16.842,642.405728,0.048697,52.375,10086.739194,0.017144,10.203,0.116224,0.005776
1954-07-01,26819.123397,6092.160954,0.012634,16.775,645.100551,0.004195,51.516,10260.639859,0.017241,10.221,0.036534,0.009413
1954-10-01,27382.792163,6194.585854,0.016813,16.697,684.643171,0.061297,51.215,10384.607891,0.012082,10.265,0.137278,0.007692
1955-01-01,28190.336450,6259.103977,0.010415,16.696,745.842631,0.089389,51.535,10506.538797,0.011742,10.323,0.082005,0.001627


## Anual

In [ ]:
# Calculando a 1 defasagem do crescimento do consumo
df_a['c_growth_lag1'] = df_a['c_growth'].shift(1)
df_a['c_growth_lag2'] = df_a['c_growth'].shift(2)

# Calculando a 1 defasagem e um adiantamento da renda
df_a['pib_lag1']  = df_a['pib'].shift(1)
df_a['pib_forw1'] = df_a['pib'].shift(-1)

# Calculando a 1 defasagem e um adiantamento do consumo
df_a['c_lag1']  = df_a['c'].shift(1)
df_a['c_lag2']  = df_a['c'].shift(2)
df_a['c_forw1'] = df_a['c'].shift(-1)

df_a['cs_lag1']  = df_a['c'].shift(1) + df_a['s'].shift(1)
df_a['cs_lag2']  = df_a['c'].shift(2) + df_a['s'].shift(2)
df_a['cs_forw1'] = df_a['c'].shift(-1) + df_a['s'].shift(-1)

df_a['cs']       = df_a['c'].shift(1)  + df_a['s']
df_a['cs_lag1']  = df_a['c'].shift(1)  + df_a['s'].shift(1)
df_a['cs_lag2']  = df_a['c'].shift(2)  + df_a['s'].shift(2)
df_a['cs_forw1'] = df_a['c'].shift(-1) + df_a['s'].shift(-1)

# Calculando a 1 e 2 defasagem e um adiantamento do sp500
df_a['sp500_r_lag1']  = df_a['sp500_ret_real'].shift(1) + 1
df_a['sp500_r_lag2']  = df_a['sp500_ret_real'].shift(2) + 1
df_a['sp500_r_forw1'] = df_a['sp500_ret_real'].shift(-1) + 1

# Calculando a 1 e 2 defasagem e um adiantamento da T-Bill
df_a['tbill_r_lag1']  = df_a['TBill_ret_real'].shift(1) + 1
df_a['tbill_r_lag2']  = df_a['TBill_ret_real'].shift(2) + 1
df_a['tbill_r_forw1'] = df_a['TBill_ret_real'].shift(-1) + 1

# Constante
df_a['const'] = 1

# Removendo os missings
dta_clean = df_a.dropna()

# Calculando as endógenas, as variáveis contemporaneas
endog_df = dta_clean[['c_forw1', 'c', 'pib_forw1', 'pib', 'tbill_r_forw1']]
exog_df  = endog_df

# Os instrumentos serão as variáveis defasadas
instrument_df = dta_clean[['tbill_r_lag1', 'pib_lag1', 'c_growth_lag1', 'const']]

# Transformando em array
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

# Vetor de zeros
endog1 = np.zeros(exog.shape[0])

In [ ]:
def moment_consumption1(params, exog):
    tau0, lammbda, tau1 = params

    c_forw1, c, pib_forw1, pib, tbill_r_forw1 = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(c_forw1/c) - tau0 - lammbda * np.log(pib_forw1/pib) - tau1 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=3)


# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res1   = mod1.fit([0.0143, 0.3096, 0.0380], maxiter=1000, inv_weights = w0inv)

### Incluindo serviços

In [ ]:
endog_df = dta_clean[['cs_forw1', 'cs', 'pib_forw1', 'pib', 'tbill_r_forw1']]
exog_df  = endog_df

endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

def moment_consumption1(params, exog):
    tau0, lammbda, tau1 = params

    cs_forw1, cs, pib_forw1, pib, tbill_r_forw1 = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(cs_forw1/cs) - tau0 - lammbda * np.log(pib_forw1/pib) - tau1 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=3)

# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res2   = mod1.fit([-0.04,3.23, -0.12], maxiter=1000, inv_weights = w0inv)

## Trimestral

In [ ]:
# Calculando a 1 defasagem do crescimento do consumo
df_q['c_growth_lag1'] = df_q['c_growth'].shift(1)
df_q['c_growth_lag2'] = df_q['c_growth'].shift(2)

# Calculando a 1 defasagem e um adiantamento da renda
df_q['pib_lag1']  = df_q['pib'].shift(1)
df_q['pib_forw1'] = df_q['pib'].shift(-1)

# Calculando a 1 defasagem e um adiantamento do consumo
df_q['c_lag1']  = df_q['c'].shift(1)
df_q['c_lag2']  = df_q['c'].shift(2)
df_q['c_forw1'] = df_q['c'].shift(-1)


df_q['cs']  = df_q['c'].shift(1)  + df_q['s']
df_q['cs_lag1']  = df_q['c'].shift(1)  + df_q['s'].shift(1)
df_q['cs_lag2']  = df_q['c'].shift(2)  + df_q['s'].shift(2)
df_q['cs_forw1'] = df_q['c'].shift(-1) + df_q['s'].shift(-1)

# Calculando a 1 e 2 defasagem e um adiantamento do sp500
df_q['sp500_r_lag1']  = df_q['sp500_ret_real'].shift(1) + 1
df_q['sp500_r_lag2']  = df_q['sp500_ret_real'].shift(2) + 1
df_q['sp500_r_forw1'] = df_q['sp500_ret_real'].shift(-1) + 1

# Calculando a 1 e 2 defasagem e um adiantamento da T-Bill
df_q['tbill_r_lag1']  = df_q['TBill_ret_real'].shift(1) + 1
df_q['tbill_r_lag2']  = df_q['TBill_ret_real'].shift(2) + 1
df_q['tbill_r_forw1'] = df_q['TBill_ret_real'].shift(-1) + 1

# Constante
df_q['const'] = 1

# Removendo os missings
dta_clean = df_q.dropna()

# Calculando as endógenas, as variáveis contemporaneas
endog_df = dta_clean[['c_forw1', 'c', 'pib_forw1', 'pib', 'tbill_r_forw1']]
exog_df  = endog_df

# Os instrumentos serão as variáveis defasadas
instrument_df = dta_clean[['tbill_r_lag1', 'pib_lag1', 'c_growth_lag1', 'const']]

# Transformando em array
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

# Vetor de zeros
endog1 = np.zeros(exog.shape[0])

In [ ]:
def moment_consumption1(params, exog):
    tau0, lammbda, tau1 = params

    c_forw1, c, pib_forw1, pib, tbill_r_forw1 = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(c_forw1/c) - tau0 - lammbda * np.log(pib_forw1/pib) - tau1 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=3)

# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res3   = mod1.fit([0.003, 0.32, 0.06], maxiter=1000, inv_weights = w0inv)

### Incluindo serviços

In [ ]:
endog_df = dta_clean[['cs_forw1', 'cs', 'pib_forw1', 'pib', 'tbill_r_forw1']]
exog_df  = endog_df
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

def moment_consumption1(params, exog):
    tau0, lammbda, tau1 = params

    cs_forw1, cs, pib_forw1, pib, tbill_r_forw1 = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(cs_forw1/cs) - tau0 - lammbda * np.log(pib_forw1/pib) - tau1 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=3)

# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res4   = mod1.fit([-0.01,3.11, -0.21], maxiter=1000, inv_weights = w0inv)

In [ ]:
print()
print( '# ----------------------------------- Anual ----------------------------------- #')
print()
print( '###### N Duraveis ######')
print()
print(res1.summary(yname='Euler Eq', xname=['Const', 'Delta Y', 'Delta r']))
print()
print( '###### N Duraveis + Servicos ######')
print()
print(res2.summary(yname='Euler Eq', xname=['Const', 'Delta Y', 'Delta r']))
print()
print()
print( '# ------------------------------- Trimestral ---------------------------------- #')
print()
print( '###### N Duraveis ######')
print()
print(res3.summary(yname='Euler Eq', xname=['Const', 'Delta Y', 'Delta r']))
print()
print( '###### N Duraveis + Servicos ######')
print()
print(res4.summary(yname='Euler Eq', xname=['Const', 'Delta Y', 'Delta r']))


# ----------------------------------- Anual ----------------------------------- #

###### N Duraveis ######

                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                    1.947e-09
Model:                 NonlinearIVGMM   Prob (Hansen J):                  1.00
Method:                           GMM                                         
Date:                Sun, 23 May 2021                                         
Time:                        17:49:09                                         
No. Observations:                  64                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Const          0.0143      0.004      3.248      0.001       0.006       0.023
Delta Y        0.3096      0.191      1.620      0.105      -0.065       0.684
Delta r        0.0380

# Problema 2) 

Usando dados da economia norte-americana estime o modelo de Shea (1989), assim descrito:

$$\Delta \ln (C_{t+1}) = \pi_0 + \pi_1 I^p_{t+1} \Delta \ln (y_{t+1}) + \pi_2 I^n_{t+1} \Delta \ln (y_{t+1}) + \pi_3 r_{t+1} + \varepsilon_{t+1}$$

em que $I^p_{t+1} = 1$ se $\Delta \ln (y_{t+1}) > 0$ e zero caso contrário; $I^n_{t+1} = 1 - I^p_{t+1}$. Teste se $\pi_1 = \pi_2$.

Considere duas frequências: i) trimestral; ii) anual.

Considere duas medidas diferentes para bens não duráveis: i) gasto com bens não duráveis; ii) gasto com bens não duráveis e serviços

In [ ]:
df_q['indicadora_p'] = (df_q['pib'] - df_q['pib'].shift(1) > 0)*1
df_a['indicadora_p'] = (df_a['pib'] - df_a['pib'].shift(1) > 0)*1

df_q['indicadora_n'] = 1-df_q['indicadora_p']
df_a['indicadora_n'] = 1-df_a['indicadora_p']

## Anual

In [ ]:
# Calculando a 1 defasagem do crescimento do consumo
df_a['c_growth_lag1'] = df_a['c_growth'].shift(1)
df_a['c_growth_lag2'] = df_a['c_growth'].shift(2)

# Calculando a 1 defasagem e um adiantamento da renda
df_a['pib_lag1']  = df_a['pib'].shift(1)
df_a['pib_forw1'] = df_a['pib'].shift(-1)

# Calculando a 1 defasagem e um adiantamento do consumo
df_a['c_lag1']  = df_a['c'].shift(1)
df_a['c_lag2']  = df_a['c'].shift(2)
df_a['c_forw1'] = df_a['c'].shift(-1)

df_a['cs_lag1']  = df_a['c'].shift(1) + df_a['s'].shift(1)
df_a['cs_lag2']  = df_a['c'].shift(2) + df_a['s'].shift(2)
df_a['cs_forw1'] = df_a['c'].shift(-1) + df_a['s'].shift(-1)

df_a['cs']       = df_a['c'].shift(1)  + df_a['s']
df_a['cs_lag1']  = df_a['c'].shift(1)  + df_a['s'].shift(1)
df_a['cs_lag2']  = df_a['c'].shift(2)  + df_a['s'].shift(2)
df_a['cs_forw1'] = df_a['c'].shift(-1) + df_a['s'].shift(-1)

# Calculando a 1 e 2 defasagem e um adiantamento do sp500
df_a['sp500_r_lag1']  = df_a['sp500_ret_real'].shift(1) + 1
df_a['sp500_r_lag2']  = df_a['sp500_ret_real'].shift(2) + 1
df_a['sp500_r_forw1'] = df_a['sp500_ret_real'].shift(-1) + 1

# Calculando a 1 e 2 defasagem e um adiantamento da T-Bill
df_a['tbill_r_lag1']  = df_a['TBill_ret_real'].shift(1) + 1
df_a['tbill_r_lag2']  = df_a['TBill_ret_real'].shift(2) + 1
df_a['tbill_r_forw1'] = df_a['TBill_ret_real'].shift(-1) + 1

# Constante
df_a['const'] = 1

# Removendo os missings
dta_clean = df_a.dropna()

# Calculando as endógenas, as variáveis contemporaneas
endog_df = dta_clean[['c_forw1', 'c', 'pib_forw1', 'pib', 'tbill_r_forw1', 'indicadora_p', 'indicadora_n']]
exog_df  = endog_df

# Os instrumentos serão as variáveis defasadas
instrument_df = dta_clean[['tbill_r_lag1', 'pib_lag1', 'c_growth_lag1', 'const']]

# Transformando em array
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

# Vetor de zeros
endog1 = np.zeros(exog.shape[0])

In [ ]:
def moment_consumption1(params, exog):
    pi0, pi1, pi2, pi3 = params

    c_forw1, c, pib_forw1, pib, tbill_r_forw1, indicadora_p, indicadora_n = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(c_forw1/c) - \
          pi0 - \
          pi1 * indicadora_p * np.log(pib_forw1/pib) - \
          pi2 * indicadora_n * np.log(pib_forw1/pib) - \
          pi3 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=4)

# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res5   = mod1.fit([0, 0, 0, 0], maxiter=1000, inv_weights = w0inv)

### Incluindo serviços

In [ ]:
endog_df = dta_clean[['pib_forw1', 'pib', 'cs_forw1', 'cs', 'tbill_r_forw1', 'indicadora_p', 'indicadora_n']]
exog_df  = endog_df
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

def moment_consumption1(params, exog):
    pi0, pi1, pi2, pi3 = params

    cs_forw1, cs, pib_forw1, pib, tbill_r_forw1, indicadora_p, indicadora_n = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(cs_forw1/cs) - \
          pi0 - \
          pi1 * indicadora_p * np.log(pib_forw1/pib) - \
          pi2 * indicadora_n * np.log(pib_forw1/pib) - \
          pi3 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=4)

# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res6   = mod1.fit([0 ,0 ,0 ,0], maxiter=1000, inv_weights = w0inv)

## Trimestral

In [ ]:
# Calculando a 1 defasagem do crescimento do consumo
df_q['c_growth_lag1'] = df_q['c_growth'].shift(1)
df_q['c_growth_lag2'] = df_q['c_growth'].shift(2)

# Calculando a 1 defasagem e um adiantamento da renda
df_q['pib_lag1']  = df_q['pib'].shift(1)
df_q['pib_forw1'] = df_q['pib'].shift(-1)

# Calculando a 1 defasagem e um adiantamento do consumo
df_q['c_lag1']  = df_q['c'].shift(1)
df_q['c_lag2']  = df_q['c'].shift(2)
df_q['c_forw1'] = df_q['c'].shift(-1)


df_q['cs']  = df_q['c'].shift(1)  + df_q['s']
df_q['cs_lag1']  = df_q['c'].shift(1)  + df_q['s'].shift(1)
df_q['cs_lag2']  = df_q['c'].shift(2)  + df_q['s'].shift(2)
df_q['cs_forw1'] = df_q['c'].shift(-1) + df_q['s'].shift(-1)

# Calculando a 1 e 2 defasagem e um adiantamento do sp500
df_q['sp500_r_lag1']  = df_q['sp500_ret_real'].shift(1) + 1
df_q['sp500_r_lag2']  = df_q['sp500_ret_real'].shift(2) + 1
df_q['sp500_r_forw1'] = df_q['sp500_ret_real'].shift(-1) + 1

# Calculando a 1 e 2 defasagem e um adiantamento da T-Bill
df_q['tbill_r_lag1']  = df_q['TBill_ret_real'].shift(1) + 1
df_q['tbill_r_lag2']  = df_q['TBill_ret_real'].shift(2) + 1
df_q['tbill_r_forw1'] = df_q['TBill_ret_real'].shift(-1) + 1

# Constante
df_q['const'] = 1

# Removendo os missings
dta_clean = df_q.dropna()

# Calculando as endógenas, as variáveis contemporaneas
endog_df = dta_clean[['c_forw1', 'c', 'pib_forw1', 'pib', 'tbill_r_forw1', 'indicadora_p', 'indicadora_n' ]]
exog_df  = endog_df

# Os instrumentos serão as variáveis defasadas
instrument_df = dta_clean[['tbill_r_lag1', 'pib_lag1', 'c_growth_lag1', 'const']]

# Transformando em array
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

# Vetor de zeros
endog1 = np.zeros(exog.shape[0])

In [ ]:
def moment_consumption1(params, exog):
    pi0, pi1, pi2, pi3 = params

    c_forw1, c, pib_forw1, pib, tbill_r_forw1, indicadora_p, indicadora_n = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(c_forw1/c) - \
          pi0 - \
          pi1 * indicadora_p * np.log(pib_forw1/pib) - \
          pi2 * indicadora_n * np.log(pib_forw1/pib) - \
          pi3 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=4)

# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res7   = mod1.fit([0, 0, 0, 0], maxiter=1000, inv_weights = w0inv)

### Incluindo serviços

In [ ]:
endog_df = dta_clean[['cs_forw1', 'cs', 'pib_forw1', 'pib', 'tbill_r_forw1', 'indicadora_p', 'indicadora_n']]
exog_df  = endog_df
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

def moment_consumption1(params, exog):
    pi0, pi1, pi2, pi3 = params

    cs_forw1, cs, pib_forw1, pib, tbill_r_forw1, indicadora_p, indicadora_n = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = np.log(cs_forw1/cs) - \
          pi0 - \
          pi1 * indicadora_p * np.log(pib_forw1/pib) - \
          pi2 * indicadora_n * np.log(pib_forw1/pib) - \
          pi3 * np.log(tbill_r_forw1)

    return -err

# Rodando o GMM. Endogenos sao 0's, exógenos as variaveis que entram na eq de Euler e instrumentos sao as variaveis defasadas
mod1   = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4, k_params=3)

# Calculando uma matriz de pesos
w0inv  = np.dot(instrument.T, instrument) / len(endog1)

# Fitando o modelo
res8   = mod1.fit([0,0,0,0], maxiter=1000, inv_weights = w0inv)

In [ ]:
print()
print( '# ----------------------------------- Anual ----------------------------------- #')
print()
print( '###### N Duraveis ######')
print()
print(res5.summary(yname='Euler Eq', xname=['Const', 'Ip Delta Y', 'In Delta Y', 'Delta r']))
print()
print( '###### N Duraveis + Servicos ######')
print()
print(res6.summary(yname='Euler Eq', xname=['Const', 'Ip Delta Y', 'In Delta Y', 'Delta r']))
print()
print()
print( '# ------------------------------- Trimestral ---------------------------------- #')
print()
print( '###### N Duraveis ######')
print()
print(res7.summary(yname='Euler Eq', xname=['Const', 'Ip Delta Y', 'In Delta Y', 'Delta r']))
print()
print( '###### N Duraveis + Servicos ######')
print()
print(res8.summary(yname='Euler Eq', xname=['Const', 'Ip Delta Y', 'In Delta Y', 'Delta r']))


# ----------------------------------- Anual ----------------------------------- #

###### N Duraveis ######

                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                    3.486e-11
Model:                 NonlinearIVGMM   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Sun, 23 May 2021                                         
Time:                        17:49:09                                         
No. Observations:                  64                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Const          0.0141      0.006      2.566      0.010       0.003       0.025
Ip Delta Y     0.3596      0.391      0.920      0.358      -0.406       1.126
In Delta Y     0.2568

Teste F é no formato

$$R \beta = 0$$

Na forma matricial:

\begin{align*}
  \begin{bmatrix}
      0\\
      0\\
      0\\
      0
  \end{bmatrix} =
  \begin{bmatrix}
      0 & 0 & 0 & 0 \\
      0 & 1 & -1 & 0 \\
      0 & 0 & 0 & 0 \\
      0 & 0 & 0 & 0
  \end{bmatrix}
  \begin{bmatrix}
      \pi_0 \\
      \pi_1 \\
      \pi_2 \\
      \pi_3
  \end{bmatrix}
  =
  \begin{bmatrix}
      0 \\
      \pi_1 - \pi_2 \\
      0 \\
      0
  \end{bmatrix}
\end{align*}

In [ ]:
from scipy import stats

regressoes = [res5, res6, res7, res8]
labels = ['Anual - Não Duravel', 'Anual - Não Duravel + Serviços',
          'Trimestral - Não Duravel', 'Trimestral - Não Duravel + Serviços']
R = np.array([
              [0, 0,  0, 0], 
              [0, -1, 1, 0], 
              [0, 0,  0, 0], 
              [0, 0,  0, 0]
              ])

for idx, res in enumerate(regressoes):
    #### TESTE F
    f_critico = res.f_test(R).fvalue[0][0] # Meu chute é que esse valor esteja errado
    p_valor_f = 1 - stats.f.cdf(f_critico, 1, len(res.resid) - 4)



    #### TESTE t
    numerador   = res.params[1] - res.params[2]
    denominador = np.sqrt(res.cov_params_default[1,1] + res.cov_params_default[2,2] - 2 * res.cov_params_default[2,1])

    # Calculando o valor critico do teste
    t_critico = numerador/denominador
    # Graus de Liberdade
    df =  len(res.resid) - 4 
    # Calculando o p-valor
    p_valor_t = 1 - stats.t.cdf(t_critico, df = df)

    # P-valor baixo rejeito H0 de igualdade
    print('# -------', labels[idx])
    print('t-crítico =', np.round(t_critico, 4) )
    print('p-valor   =', np.round(p_valor_t, 4) )
    print()
    print('F-crítico =', np.round(f_critico, 4) )
    print('p-valor   =', np.round(p_valor_f, 4) )
    print()
    print()

# ------- Anual - Não Duravel
t-crítico = 0.1915
p-valor   = 0.4244

F-crítico = 0.0367
p-valor   = 0.8488


# ------- Anual - Não Duravel + Serviços
t-crítico = -1.5827
p-valor   = 0.9406

F-crítico = 2.5049
p-valor   = 0.1187


# ------- Trimestral - Não Duravel
t-crítico = -0.0634
p-valor   = 0.5253

F-crítico = 0.004
p-valor   = 0.9495


# ------- Trimestral - Não Duravel + Serviços
t-crítico = 0.0574
p-valor   = 0.4771

F-crítico = 0.0033
p-valor   = 0.9543




In [ ]:
print(res8.summary())

# Grau de Liberdade
df0 = len(res8.resid) - 4

# Valor critico e p-valor
t_critico = res8.params[1] / np.sqrt(res8.cov_params_default[1,1] )
p_value_t = 1 - stats.t.cdf(t_critico, df = df0)

# Calculando manualmente o teste
print(np.round(t_critico, 3), np.round(p_value_t, 10))

                            NonlinearIVGMM Results                            
Dep. Variable:                      y   Hansen J:                    1.122e-10
Model:                 NonlinearIVGMM   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Sun, 23 May 2021                                         
Time:                        17:49:09                                         
No. Observations:                 265                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x4            -0.0324      0.607     -0.053      0.957      -1.221       1.157
x5             6.0171     89.608      0.067      0.946    -169.611     181.645
x6           -19.0140    346.911     -0.055      0.956    -698.946     660.918
x7             0.3651      4.994      0.073      0.9